In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..", "..")))
from nlp.preprocessing import bert_jamo_spliter
import json

In [2]:
r = open("../../datasets/extractive_summary/train.jsonl", "r", encoding="utf8")
w = open("../../datasets/extractive_summary/train_hg.json", "a", encoding="utf8")
c = open("../../datasets/extractive_summary/corpus.txt", "a", encoding="utf8")
try:
    count = 0
    while True:
        line = r.readline()
        if not line: break
        count += 1
        line = json.loads(line)
        for i, s in enumerate(line["article_original"]):
            line["article_original"][i] = bert_jamo_spliter.encode(s)
            c.write(line["article_original"][i] + "\n")
        line["abstractive"] = bert_jamo_spliter.encode(line["abstractive"])
        c.write(line["abstractive"] + "\n")
        w.write(json.dumps(line, ensure_ascii=False))
        if count % 10000 == 0:
            print(f"█{count}")
        elif count % 100 == 0:
            print("█", end="")
    print(f"{count}")
except Exception as e:
    raise e
finally:
    r.close()
    w.close()
    c.close()

████████████████████████████████████████████████████████████████████████████████████████████████████10000
████████████████████████████████████████████████████████████████████████████████████████████████████20000
████████████████████████████████████████████████████████████████████████████████████████████████████30000
████████████████████████████████████████████████████████████████████████████████████████████████████40000
████████████████████████████42803


In [3]:
with open("../../datasets/extractive_summary/corpus.txt", "r", encoding="utf8") as c:
    dataset = c.readlines()

len(dataset)

610225

### Make WordPiece tokenizer

https://velog.io/@nawnoes/Huggingface-tokenizers%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%9C-Wordpiece-Tokenizer-%EB%A7%8C%EB%93%A4%EA%B8%B0

In [4]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [5]:
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents

bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

In [6]:
from tokenizers.pre_tokenizers import Whitespace

bert_tokenizer.pre_tokenizer = Whitespace()

In [7]:
from tokenizers.processors import TemplateProcessing

bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)

In [8]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=30522, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)
files = ["../../datasets/extractive_summary/corpus.txt"]
bert_tokenizer.train(files, trainer)

bert_tokenizer.save("../../datasets/extractive_summary/wpm.json")

In [9]:
output = bert_tokenizer.encode(bert_jamo_spliter.encode("당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것으로 추정되는 문자와 관련해 당진시의회가 행정사무조사특별위원회를 구성해 조사하겠다고 밝혔다."))
print(output.tokens)
# [1, 27253, 16, 93, 11, 5097, 5, 7961, 5112, 6218, 0, 35, 2]

recover = bert_tokenizer.decode(output.ids)
recover = recover.replace('#', '')
print(bert_jamo_spliter.decode(recover))
# "Hello , y ' all ! How are you ?"

['[CLS]', 'ㄷㅏㅇㅈㅣㄴㅅㅣ', 'ㅁㅜㄴㅎㅘㄱㅘㄴㄱㅘㅇ', '##ㄱㅘㄹㅡㄹ', 'ㄷㅐㅅㅏㅇㅇㅡㄹㅗ', 'ㅎㅏㄴㅡㄴ', 'ㅎㅐㅇㅈㅓㅇㅅㅏㅁㅜㄱㅏㅁㅅㅏㅇㅔㅅㅓ', 'ㄷㅏㅇㅈㅣㄴㅅㅣㄹㅣㅂ', '##ㅎㅏㅂㅊㅏㅇㄷㅏㄴ', 'ㄱㅘㄴㄱㅖㅈㅏㄱㅏ', 'ㅂㅗㄴㅐㄴ', 'ㄱㅓㅅㅇㅡㄹㅗ', 'ㅊㅜㅈㅓㅇㄷㅚㄴㅡㄴ', 'ㅁㅜㄴㅈㅏ', '##ㅇㅘ', 'ㄱㅘㄴㄹㅕㄴㅎㅐ', 'ㄷㅏㅇㅈㅣㄴㅅㅣㅇㅢㅎㅚ', '##ㄱㅏ', 'ㅎㅐㅇㅈㅓㅇㅅㅏㅁㅜ', '##ㅈㅗㅅㅏ', '##ㅌㅡㄱㅂㅕㄹㅇㅟㅇㅝㄴㅎㅚ', '##ㄹㅡㄹ', 'ㄱㅜㅅㅓㅇㅎㅐ', 'ㅈㅗㅅㅏ', '##ㅎㅏㄱㅔㅆㄷㅏㄱㅗ', 'ㅂㅏㄺㅎㅕㅆㄷㅏ', '.', '[SEP]']
당진 문화관 과 대상으 하 행정사무감사에 당진시 합창 관계자 보 것으 추정되 문  관련 당진시의  행정사 조 특별위원  구성 조 하겠다 밝혔 .


In [2]:
bert_jamo_spliter.decode("ㅁㅜㄴㅎㅘㄱㅘㄴㄱㅘㅇ")

'문화관광'